In [51]:
from math import *
from tqdm import tqdm
import numpy as np
from PIL import Image

In [52]:
class sphere():
    def __init__(self, ce, r, co, spe, refl, fuz, spht = 'normal'):
        self.spheretype = spht
        self.center = ce
        self.radius = r
        self.color = co
        self.specular = spe
        self.reflective = refl
        self.fuzzy = fuz

spheres = []

spheres.append(sphere(np.array([0.8, 0, 3]), 0.5, np.array([0, 255, 255]), -1, 0, 0))#blue
spheres.append(sphere(np.array([0, 0, 6]), 1, np.array([238, 130, 238]), 10, 0.3, 0)) #pink
spheres.append(sphere(np.array([0, -10, 4.5]), 9, np.array([80, 200, 120]), 25, 0.3, 1))#green
spheres.append(sphere(np.array([0.8, 1.2, 5.5]), 0.1, np.array([255, 255, 255]), 0, 0, 0,'spherelight'))

def IntersectRaySphere(O, d, sphere):
    center = sphere.center
    radius = sphere.radius
    oc = O - center

    k1 = np.sum(np.dot(d, d))
    k2 = 2*np.sum(np.dot(oc, d))
    k3 = np.sum(np.dot(oc, oc)) - radius*radius

    discriminant = k2*k2 - 4*k1*k3
    if discriminant < 0:
        return inf, inf

    t1 = (-k2 + sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - sqrt(discriminant)) / (2*k1)
    return t1, t2

def ClosestIntersection(O, d, tmin, tmax):
    closest_t = inf
    closest_sphere = None
    
    for sphere in spheres:
        t1, t2 = IntersectRaySphere(O, d, sphere)
        if t1 >= tmin and t1 <= tmax and t1 < closest_t:
            closest_t = t1
            closest_sphere = sphere
        if t2 >= tmin and t2 <= tmax and t2 < closest_t:
            closest_t = t2
            closest_sphere = sphere
    return closest_sphere, closest_t

def TraceRay(O, d, tmin, tmax, depth):
    closest_sphere, closest_t = ClosestIntersection(O, d, tmin, tmax)
        
    if closest_sphere == None:
        return BACKGROUND_COLOR
    
    P = O + np.dot(closest_t, d)  # вычисление пересечения
    N = P - closest_sphere.center  # вычисление нормали сферы в точке пересечения
    N = N / sqrt(np.sum(np.square(N)))
    if closest_sphere.spheretype == 'spherelight':
        return closest_sphere.color 
    local_color = closest_sphere.color * Lighting(P, N, -d, closest_sphere.specular)

    # предел рекурсии или не отражающий объект 
    r = closest_sphere.reflective
    if depth <= 0 or r <= 0:
        return local_color

    # вычисление отражённого цвета
  
    if closest_sphere.fuzzy == 0:
        R = ReflectRay(-d, N)
        reflected_color = TraceRay(P, R, 0.001, inf, depth - 1)
    else:
        reflected_color = np.array([0, 0, 0])
        R1 = ReflectRay(-d, N + np.array([0.02, 0, 0]))
        R2 = ReflectRay(-d, N + np.array([0, 0.02, 0]))
        R3 = ReflectRay(-d, N + np.array([0, 0, 0.02]))
        R4 = ReflectRay(-d, N + np.array([0.02, 0.02, 0]))
        R5 = ReflectRay(-d, N + np.array([0.02, 0, 0.02]))
        R6 = ReflectRay(-d, N + np.array([0, 0.02, 0.02]))
        R7 = ReflectRay(-d, N + np.array([0.02, 0.02, 0.02]))
        R8 = ReflectRay(-d, N)
        reflected_color = (TraceRay(P, R1, 0.001, inf, 0)*pow(np.sum(np.dot(R1, N))/(sqrt(np.sum(np.square(R1)))*sqrt(np.sum(np.square(N)))), 2) + 
                           TraceRay(P, R2, 0.001, inf, 0)*pow(np.sum(np.dot(R2, N))/(sqrt(np.sum(np.square(R1)))*sqrt(np.sum(np.square(N)))), 2) + 
                           TraceRay(P, R3, 0.001, inf, 0)*pow(np.sum(np.dot(R3, N))/(sqrt(np.sum(np.square(R1)))*sqrt(np.sum(np.square(N)))), 2) + 
                           TraceRay(P, R4, 0.001, inf, 0)*pow(np.sum(np.dot(R4, N))/(sqrt(np.sum(np.square(R1)))*sqrt(np.sum(np.square(N)))), 2) + 
                           TraceRay(P, R5, 0.001, inf, 0)*pow(np.sum(np.dot(R5, N))/(sqrt(np.sum(np.square(R1)))*sqrt(np.sum(np.square(N)))), 2) + 
                           TraceRay(P, R6, 0.001, inf, 0)*pow(np.sum(np.dot(R6, N))/(sqrt(np.sum(np.square(R1)))*sqrt(np.sum(np.square(N)))), 2) + 
                           TraceRay(P, R7, 0.001, inf, 0)*pow(np.sum(np.dot(R7, N))/(sqrt(np.sum(np.square(R1)))*sqrt(np.sum(np.square(N)))), 2) + 
                           TraceRay(P, R8, 0.001, inf, 0)*pow(np.sum(np.dot(R8, N))/(sqrt(np.sum(np.square(R1)))*sqrt(np.sum(np.square(N)))), 1))

    return local_color*(1 - r) + reflected_color*r


In [53]:
#освещение

class light():
    def __init__(self, t, i, d = np.array([0, 0, 0])):
        self.typelight = t
        self.intensity = i
        self.direction = d

lights = []
lights.append(light('ambient', 0.2))
lights.append(light('point', 0.52, np.array([0.75, 1.25, 5.55])))
lights.append(light('point', 0.07, np.array([0.85, 1.25, 5.55])))
lights.append(light('point', 0.07, np.array([0.75, 1.15, 5.55])))
lights.append(light('point', 0.07, np.array([0.85, 1.15, 5.55])))
lights.append(light('point', 0.07, np.array([0.75, 1.25, 5.45])))
lights.append(light('point', 0.07, np.array([0.85, 1.25, 5.45])))
lights.append(light('point', 0.07, np.array([0.75, 1.15, 5.45])))
lights.append(light('point', 0.07, np.array([0.85, 1.15, 5.45])))

def Lighting(P, N, V, s):
    i = 0.0
    for light in lights:
        if light.typelight == 'ambient':
            i += light.intensity
        else:
            if light.typelight == 'point':
                L = light.direction - P
                tmax = 1
                
            # Проверка тени
            shadow_sphere, shadow_t = ClosestIntersection(P, L, 0.001, tmax)
            
            if  shadow_sphere == None or shadow_sphere.spheretype == 'spherelight': 
                # матовые
                n_dot_l = np.sum(np.dot(N, L))
                if n_dot_l > 0:
                    i += light.intensity*n_dot_l/(sqrt(np.sum(np.square(N))) * sqrt(np.sum(np.square(L))))

                # зеркальные
                if s != -1:
                    R = ReflectRay(L, N)
                    r_dot_v = np.sum(np.dot(R, V))
                    if r_dot_v > 0:
                        i += light.intensity * pow(r_dot_v/(sqrt(np.sum(np.square(R)))*sqrt(np.sum(np.square(V)))), s)
    return i

def ReflectRay(R, N):
    return 2*N*np.sum(np.dot(N, R)) - R;

In [54]:
O = np.array([0,0,0])
recursion_depth = 1

image = Image.open(r'canvas.png')

ImageW, ImageH = image.size
dist = 10
WindowW = 10
WindowH = 10
BACKGROUND_COLOR = np.array([0, 0, 0])

def CanvasToView(x,y):
    return np.array([x*WindowW/ImageW, -y*WindowH/ImageH, dist])

for x in tqdm(range(-300, 300)):
    xx = x + 300
    for y in range(-300, 300):
        yy = y + 300
        color = TraceRay(O, CanvasToView(x, y), 0.001, inf, recursion_depth)
        image.putpixel((xx, yy), tuple(int(c) for c in color))
image.show()

100%|██████████| 600/600 [10:12<00:00,  1.02s/it]
